# **Lab 3**


## Tabular Data with Pandas

> **Credit note**: A portion of this lab tutorial was motivated by the nice examples from https://github.com/guipsamora and University of Berkeley's [Data 100](https://github.com/DS-100/su19/) (Summer 2019 version) lecture notebooks.


Last week, we had our first attempt at using **Pandas**, a powerful Python library written for the purpose of data manipulation and analysis. This week, we revisit Pandas in a more detailed manner, particularly on some basic concepts in handling and accessing Dataframes, how to perform sorting and counting of values in the Dataframe and also grouping data based on selected criteria. 

OK, let's import some important packages that we may need later.

In [ ]:
import numpy as np
import pandas as pd

### Preamble

First, let's learn a few interesting commands and functions in Python that could come in handy sometimes. The command `%whos` allows us to see what are the active variables loaded in the memory. 

In [ ]:
pi = 3.142
twopi = 2*3.142
a_tray_of_pi = np.array([twopi, pi, twopi, pi], dtype='int64')  # dtype fixes the data type of choice

In [ ]:
%whos

Use `del` to delete variables of your choice:

In [ ]:
del twopi

In [ ]:
%whos

You can also erase all variables from memory using another magic function `%reset`. *Magic* functions are exclusive to the IPython kernel (note: not Python) and they perform special functions that usually involve the IPython kernel or to interact with the operating system.

In [ ]:
%reset

IPython's `%timeit` magic function is useful to perform speed benchmarking of Python codes. It runs an operation in $N$ number of loops for $T$ number of times, then takes the best of the $T$ rounds. It reports back how much time was consumed per loop. 

Let's create 30 million integers and put them into a list and dictionary.

In [ ]:
biglist = []
bigdict = {}
for j in range(30000000):
    biglist.append(j)     # recall: this is how you add to a list
    bigdict[j]=j          # this is how you add to a dictionary

In [ ]:
%timeit 9999999 in biglist

In [ ]:
%timeit 9999999 in bigdict

Which data structure is quicker to access?

<br />

To know the package versions, call the common method `__version__`:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.__version__       # double underscore on both sides

In [ ]:
np.__version__

### Working with tabular data

Let's use another dataset (something more substantial and informative than hospital locations) to put Pandas to some practice. The [Internet Movie DataBase (IMDB)](https://www.imdb.com/) is a treasure trove of all kinds of movies, which comes with a large range of metadata (year of release, actors, description) as well as critic ratings. The data which was taken from [**data.world**](https://data.world/studentoflife/imdb-top-250-lists-and-5000-or-so-data-records) has been provided. 

To start, load the dataset using pandas' nifty `read_csv` function. It's a really powerful function which is also easy to use:

In [ ]:
movies = pd.read_csv("IMDB_top250.csv")
movies

IPython displays the contents of the read dataframe in brief form. 

If you are using Jupyter Notebook, you will notice that there are a lot more columns not displayed here (double-check this with the data opened in spreadsheet). The three dots, "..." in the column name area shows that there are some column data suppressed from being shown. 

Commonly known properties such as `shape` and `size` still work 

**Tip**: Sometimes you can make a wild guess and still get the function or property correct! 

In [ ]:
movies.shape

In [ ]:
movies.size

In [ ]:
movies.columns

The function `describe` performs some statistical analysis over all numeric columns (noticed that the non-numeric values are left out). Note that this is can look really easy to use, but it can also be quite misleading if you do not know what's going on.


In [ ]:
movies.describe()

For example, what is the "mean" of the Year column? Does it mean anything? What about the "mean" of the imdbRating column?

The `head` and `tail` command shows only a few lines near the top or bottom of the data. This is kinda useful if you only want a glimpse of the data and not show everything at once.

In [ ]:
movies.head(7)   # first 7 rows

In [ ]:
movies.tail()  # 5 by default

If you examine the data, it appears that the first column (which is unnamed) could be redundant if you already can have the index number. It is also some kind of rank number for the top 250 movies which have already been sorted based on imDBRating. However, if there's a tie in imDBRating, it is not clear what other criteria is used to sort thereafter. So, using this number as the rank is also not accurate. Let's drop that column.

In [ ]:
movies.drop(columns=['Unnamed: 0'], inplace=True)

Many functions in pandas come with the flag `inplace` that has been set to False. This prevents you from making costly mistakes or errors. Setting `inplace=True` ensures that the change is made on the dataframe itself.

In [ ]:
movies.head()

To change the index of the the dataframe to the imdbID (ID of the movie entry), you can either re-read the entire dataset:

In [ ]:
movies2 = pd.read_csv("IMDB_top250.csv", index_col="imdbID")
movies2.head()

Or, you can just use `set_index` anytime, if you had already read the data earlier

In [ ]:
movies3 = movies.set_index("imdbID")
movies3.head()

The original dataframe `movies` did not change because `inplace` was not set to True. So, we have modified the index in the new dataframe `movies3`.

In [ ]:
movies.head()

In [ ]:
movies3.index

Indices can have names. Since the new index for `movies3` came from an existing column read earlier, the index now has a name.

In [ ]:
movies3.index.name

> **Note**: Column names must be unique. If we try to read in a file for which the column names are not unique, Pandas will automatically remove any duplicates.

### Dataframe indexing

The DataFrame class has an indexing operator [] that lets you do a variety of different things. If your provide a String to the [] operator, you get back a Series corresponding to the requested label.

In [ ]:
movies["Title"].head(6)

If you want the data to be retained in a Dataframe form instead of Series, add the `to_frame()` function:

In [ ]:
movies["Title"].head(6).to_frame()

If you want the movie titles in an array, you can use the property `values` which extracts out the exact value in that column, without all the dataframe frills like indices or other information like Name and dtype.

In [ ]:
movies["Title"].values

The [] operator also accepts a list of Strings which in that case, you will be getting back a dataframe containing the columns that you specify:

In [ ]:
movies[["Title","imdbRating"]]

Print it again with the `values` property. Notice how the multi-column data is now organized in the array...

In [ ]:
movies[["Title","imdbRating"]].values

The [] operator also accepts numerical slices as arguments. In this case, we are indexing by row, not column!

In [ ]:
movies[0:3]

An interesting behavior happens if you try to give it a single integer index instead of a slice. It doesn't work because it tries to use it like a name instead...

In [ ]:
movies[3]

In [ ]:
movies[["3"]]

### Accessing cells with loc and iloc

To access cells, the function `loc` is most handy. 

You can access a particular row by name (that is, by the index). So, using `movies3` which we had replaced the original index with the imdbID, we can now extract based on these IDs.

In [ ]:
movies3.loc["tt0068646"]

In [ ]:
movies.loc[1]

`loc` also supports slicing (for all types, including numerics and string labels!). Note that slicing for `loc` is inclusive even for numeric slices. 

In [ ]:
movies.loc[0:4, 'Title':'Director']

You can make specific row or column selections, by doing it in a list way.

In [ ]:
movies.loc[[0, 1, 2, 3, 4], ['Title','Year', 'Runtime']]

If we provide only a single label for the row or column argument, we get back a Series.

In [ ]:
movies.loc[0:4, "Title"]

In [ ]:
movies.loc[4, "Title":"Runtime"]

`iloc` is very similar, but it is used to access *numerical positions* instead of label names. So this is similar to what we are familiar with when accessing arrays. For example, to access to the top 3 rows and top 3 columns of a table, we can use [0:3, 0:3]. `iloc` slicing is therefore exclusive, just like standard Python slicing of numerical values.

In [ ]:
movies.iloc[0:3, 0:3]

This could be more intuitive at a glance, but generally, `loc` is preferred for a number of reasons:
1. It is harder to make mistakes since you have to literally write out what you want to get.
2. Code is easier to read, because the reader doesn't have to know e.g. what column #31 represents.
3. It is robust against permutations of the data, e.g. the order of two columns are switched at the data source.

However, if necessary, using `iloc` is OK as well.

### "Filters" with Boolean indexing

Just like arrays, Boolean indexing is also possible. In fact, they can be quite powerful as boolean values can be generated using expressions that evaluate to True/False. In this sense, we are creating "filters", which can be used to extract only specific data (rows) that fulfills the criteria set.

Let's create a filter, which we want to obtain movies that were released later than year 2010.

In [ ]:
movies["Year"] > 2010

We can pass this filter as a list of indices (or a slice if you wish) of the DataFrame.

In [ ]:
filter1 = movies["Year"] > 2010
movies[filter1]

**Q1**: Which movie of the Drama genre, released after 2010, managed to obtain a rating of more than 8.5 on IMDB? 

In [ ]:
# complete the code
filter2 = 


### Sorting data

Sorting data is one of the most meaningful things that we need do when it comes to data. Without sorting, it may be difficult to perform ranking on the data (or specific columns of the data) and a whole lot of other tasks.

Let's now create a copy of the Dataframe sorted by a specific column using `sort_values` function.

In [ ]:
movies.sort_values('Year')

What if we want to see the latest years first? To sort by descending order, set `ascending=False`:  

In [ ]:
movies.sort_values('Year', ascending=False)

In [ ]:
movies4 = movies[["Title", "Year", "imdbRating"]]   #  take only these 3 columns
movies4.sort_values(['Year', 'imdbRating'], ascending=[False, False])    # sort by two criteria: Year, then imdbRating

Note: If you have extracted the data out from the DataFrame into *Series* object, you can still use `sort_values`.

### Counting unique values

Another useful function is `value_counts`. It gives the total count of specific unique values of a column. This is good if we intend to construct [histograms](https://en.wikipedia.org/wiki/Histogram) to represent the distribution of data.

In [ ]:
vc = movies["Year"].value_counts()
vc

Indices can be sorted as well. If you look at the Series created above, the values are the number of movies by the year. We cannot use `sort_values` which is only applicable to values. To sort indices, use `sort_index` instead:

In [ ]:
vcis = vc.sort_index()
vcis

### Simple charts 

Let's try to plot the information above. A bar chart would be a good choice, since we want to show the spread of all-time top 250 movies (as far as IMDB is concerned) in chronological order, i.e. according to year. 

Python has a classic plotting library called [**matplotlib**](https://matplotlib.org/) which was adapted from Matlab's plotting tools and functions. It is versatile, and highly customisable, but can also be a little too "low-level" for some peoples' liking. Some data scientists use this to have greater control over plots, although many also prefer more modern visualization libraries such as [**seaborn**](https://seaborn.pydata.org/) and [**bokeh**](https://docs.bokeh.org/en/latest/index.html).

In this lab, you will explore a little on **matplotlib**. We will use **seaborn** in some upcoming labs later.

To use matplotlib, import the package as follows:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib as mpl            # import main library to
mpl.style.use('default')            # set visualization style

To plot in matplotlib, a few lines of code are needed. 

In [ ]:
plt.figure(figsize=(15,3))                 # this adjusts the size of the figure displayed
plt.bar(vcis.index, vcis)         # this plots the bar plot, first parameter: x-axis values, second: y-axis values
plt.xlabel('Year')                         # label on x axis 
plt.ylabel('# of movies')                  # label on y axis
plt.title('Number of Top-250 movies in IMDB by Year')      # title of figure
plt.show()                                 # this line is necessary to display out the entire figure

**Q2**: Based on IMDB's top-250 list of movies, who is the most successful director?

In [ ]:
# fill in code


**Q3**: In which movies in the top-250 list, did the following actors/actresses appear? 
* Morgan Freeman
* Emma Stone
* Russell Crowe

In [ ]:
# fill in code


> **Note**: You may think that it would be easier to just search for these answers directly from the data file (csv), but think of the case where the data would be very large and it may be challenging to search manually. Doing this in code also allows us to write scripts or interfaces with applications and services.

### Grouping

Grouping is the next concept that we can apply to data. It is intuitive (almost human nature) to put ideas and concepts into groups when there is an 'overload' of data that we can digest at one go. When data is big, it is difficult to examine it thoroughly to form conclusions based on the entire data. For example, even when describing data with statistics, it can sometimes be misleading or inaccurate to simply take the mean of the entire data, unless of course we intend to take the mean for some purpose/task such as normalization. It gives a better context if we were to take the mean of smaller, distinct groups so that we can summarise the behavior of different groups within the same data. 

Pandas has a powerful `groupby` function to do just that.

Now, this is incomplete. The object has been created but we also need to specify *how* the grouping is to be executed. Think of it this way: If a group contains 10 data entries, the 10 data entries should be summarised into a single value so that the new DataFrame contains this value representing that particular group.

Let's choose the action to be the mean or average.

In [ ]:
movies.groupby('Production').mean(numeric_only=True)

And so we have applied mean on these groups. Some columns make sense after applying mean, some don't. The mean year does not really give us anything informative. The mean ratings do. Notice that when we applied `mean()`, again only the numeric data is left. Let's trim the Dataframe further...

In [ ]:
gbp = movies.groupby('Production').mean(numeric_only=True)
gbp[["Metascore", "imdbRating"]]         # take only these 2 columns

To get it sorted, append some code *after* it...   

> Pandas is very powerful, if you want to do something after your existing line of code, keep adding code behind it. Most of the time, it works!

In [ ]:
gbp[["Metascore", "imdbRating"]].sort_values('imdbRating', ascending=False)

We can also calculate more than one type of value per group. For instance, we can calculate the min and max of each group:

In [ ]:
movies1 = movies.drop(['Released', 'Writer', 'Awards', 'DVD', 'BoxOffice', 'Website'], axis=1)
movies1.groupby('Production').agg(['min', 'max'])

Oh well, it did what we ask it to do! Each column has now min and max calculated. Now, this is looking a bit silly and off track. We are only interested in the min and max of imdbRating, not everything. 

Pandas allows *column names* to be used as property fields. Watch how `imdbRating` is used as a property field.

In [ ]:
movies.groupby('Production').imdbRating.agg(['min', 'max'])

`groupby` can also perform multi-level grouping, where we can first group by one criteria (column), and then for each of these groups, further group based on another criteria (column).

In [ ]:
gbpr = movies.groupby(['Production', 'Rated']).imdbRating.agg('mean')
gbpr

## Additional Exercises

**AE1**: Going back to the IMDB data, notice there there is quite a bit of text information in the table that we did not use? Processing and understanding text data is a whole topic by itself, we will not attempt to do very sophisticated interpretations of the text data. However, we can use some simple text parsing to process and extract some valuable information for these movies. 

If you observe the *Awards* column, we can find text such as "Nominated for 7 Oscars", "Won 7 Oscars", etc. on a quite consistent format. Write some code to process these text. Extract out 
* Number of Oscar wins
* Number of Oscar nominations (did not win)
and create two new columns in this table containing these values. Some movies won many other awards, but we have to ignore them due to limited information of what awards are those.

In [ ]:
# work on it here
